In [1]:
import pandas as pd
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
in_file = "../features/UCM-ResNet-50.csv"
out_file = "../results/" + in_file.split("/")[-1].split(".csv")[0] + ".npy"
df = pd.read_csv(in_file)

In [3]:
df.head()

Unnamed: 0         0         1         2         3         4  \
0        overpass42.tif -3.452254 -2.115298  2.337132 -4.735918 -0.864968   
1  mobilehomepark75.tif -3.502577 -2.483521 -1.433787 -4.362304 -1.102523   
2     tenniscourt46.tif -5.429916 -0.219190 -2.328792 -3.109982 -1.811881   
3        overpass29.tif -6.092787 -0.592579 -0.126701 -3.286357 -2.168131   
4       chaparral41.tif -3.790862 -0.265185 -2.819205  0.805453 -1.364472   

          5         6         7         8    ...          990       991  \
0 -0.804796  0.590069 -1.956671 -1.481739    ...    -4.031911 -5.490343   
1 -1.828413 -2.268677 -1.050013 -1.267096    ...    -0.171323 -1.107236   
2 -2.556363 -2.679789  1.646758 -0.893228    ...    -1.462789 -1.348120   
3 -0.803777 -1.341849 -0.262573 -1.415286    ...    -3.785638 -3.402800   
4  6.193651  5.856352 -1.982988 -1.174672    ...     2.636379  5.201247   

        992       993       994       995       996       997       998  \
0 -4.604525 -7.912472 -5.986009 -3.889988 -2.727010 -5.560139 -1.653060   
1 -1.400606 -2.284436 -0.279304 -1.436117  1.258421 -1.741745 -1.003503   
2 -2.905115 -3.428840 -4.179016 -2.366599 -1.274371 -4.796336 -1.585423   
3 -3.301758 -4.492131 -3.419818 -4.899810 -2.095596 -3.085959 -0.987355   
4  3.033085 -0.432375  3.056822  0.599141  1.257810  0.735891  0.825257   

        999  
0 -0.281703  
1  0.169005  
2  1.691911  
3  2.889485  
4  3.249267  

[5 rows x 1001 columns]

In [4]:
X = np.array(df.iloc[:,1:])

import re
def getdigit(text):
    return re.compile('\d+').findall(text)
labels = [i.split(getdigit(i)[0])[0] for i in df.iloc[:,0]]

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(labels)

print "feature data is shape", X.shape
print "label data is shape", y.shape

feature data is shape (2100, 1000)
label data is shape (2100,)


In [5]:
pipe_svc = Pipeline([('sc1', StandardScaler()),
                    ('clf', SVC(kernel='linear'))])

param_range = np.logspace(-10, 1, 100)
param_grid = [{'clf__C': param_range}]
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, cv=10, n_jobs=-1)

In [6]:
skf = StratifiedKFold(y, n_folds=5)


In [7]:
scores = []
for train_index, test_index in skf:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gs.fit(X_train, y_train)
    clf = SVC(kernel= 'linear', C = gs.best_params_.values()[0])
    clf.fit(X_train, y_train) 
    scores.append(clf.score(X_test, y_test))

In [8]:
scores

[0.95714285714285718,
 0.94285714285714284,
 0.94285714285714284,
 0.92142857142857137,
 0.95238095238095233]

In [9]:
np.mean(scores)

0.94333333333333313

In [10]:
gs.best_params_.values()[0]

0.00278255940220712

In [11]:
np.save(out_file, scores)